# Scowl Simulated Deployment

The objective of this notebook is to:
1. Define the goals of the simulation run
2. Declare the methods of running the simulation
3. Visualize the simulation parameters
4. Run the simulations
5. Process the log data
6. Visualize the results

## What is the scope of the simulation?

1. Confirm the simulation parameters
    1. `Peer` mix (generator, consumer, superpeer/storage)
    2. Supply/Demand ratio
    3. `Peer` to `Tracker` network latency
    
2. The evaluation should answer:
    1. How many peers were rebalanced (independant variable) and how fast was the tracker able to rebalance its peers (dependant variable – in Hz)?
    2. What was the limiting resource? (e.g., network, cpu, memory, etc.)
    3. At what scale of peers did the limiting factor occur?
    4. How long was the average peer "starved" for electricity?
    5. When demand exceeded supply, did a black out occur?
    6. When should the system be sharded?
    7. When the network partitioned, did peers operating in SOC mode cause a black out in their region?
    8. *Stretch Goal* How small of a partition can the grid operate under?

# Simulation "Run Sheet"

The simulation should have (2) main parts:
1. **Electrical stress test under a healthy network** – There are imbalances between supply and demand, but the network remains up. `Trackers` attempt to pair `Consumers` with `Generators` that have additional capacity, the goal is to re-build the *ephemeral distribution trees* as fast as possible, so that `Consumer` starvation is minimized.
2. **Electrical stress test under a partitioned network** – Communication between the tracker and the peers is broken. Each `Consumer` is operating as a self optimizing consumer (soc). Each `Generator` is a self-optimizing generator (sog). `Generators` can **only** warn SOCs of an impending black out by sending a `capacity_low` rpc to their partition. This is analagous to real-world electricity consumers feeling the voltage sag of a brown-out, in advance of a black out. The warned consumers respond by self-imposing a call to `DemandResponse()`, until the `Generator` unchokes them <sup>1</sup>. The key metric here is how many `capacity_low` messages are sent, and how much *deferred consumption* consumers experience (i.e., this messaged in time and kWhs).

The energy mix will be based on the current the energy mix of New York State, and renewables will experience the same fluctations. The same "day" will be simulated for both the healthy network day (scowl) and the partitioned network day (soc/sog).

***
<sup>1</sup> This behaviour mimics the TCP congestion control algorithm, with a similar expectation that backing off demand will result in data (electricity) flowing again.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Electricity Generation Mix NY State (2020)

In [ ]:
g22 = pd.read_csv("data/2022_elec_gen_mix.csv")
g22.set_index("type", inplace= True)
g22['portion'] = g22.mwh / (g22.mwh.sum())
g22.sort_values('mwh', ascending=False, inplace=True)
g22['renewable'] = [False, False, True, True, True, False, True, True]
g22

In [ ]:
# pd.concat([g22,pd.DataFrame(g22.sum()).T]).rename(index={0: '2022 Total'})

In [ ]:
# Assigning consistent colors for graphs
import matplotlib.colors as mcolors
mcolors.TABLEAU_COLORS

colors = []
for i, v in enumerate(g22.index):
    colors.append([v, list(mcolors.TABLEAU_COLORS.items())[i][0]])
colors = pd.DataFrame(colors, columns=["type", "color"]).set_index("type")

# using either is fine
g22 = pd.concat([g22,colors], axis = 1)
color_dict = g22.color.to_dict()

***

## Electricity Generation Mix NY State (2030)

In [ ]:
g30 = pd.read_csv("data/2030_elec_gen_mix.csv")
g30.set_index("type", inplace= True)
g30['portion'] = g30.mwh / (g30.mwh.sum())
g30.sort_values('mwh', ascending=False, inplace=True)
g22 = g22.reindex(g30.index.to_list()) # make g22 use same index order as g30
g30['renewable'] = g22['renewable']
g30['color'] = g22['color']
g30
# pd.concat([g30,pd.DataFrame(g30.sum()).T]).rename(index={0: '2023 Total'})

## Electricity Mix 2022 and 2030

In [ ]:
fig = plt.figure(figsize=(7.5,3), dpi=110)

ax1 = fig.add_subplot(121)
ax1.pie(np.round(g22[g22.portion > 0].portion*100,1), labels=g22[g22.portion > 0].index,autopct='%.1f%%', textprops={'fontsize': 8}, colors=g22[g22.portion > 0].color)
ax1.title.set_text('Electricity generation by source\nNew York State (2022)')
ax1.title.set_size(9)

ax2 = fig.add_subplot(122)
ax2.pie(np.round(g30[g30.portion > 0].portion*100,1), labels=g30[g30.portion > 0].index,autopct='%.1f%%', textprops={'fontsize': 8}, colors=g30[g30.portion > 0].color)
ax2.title.set_text('Electricity generation by source\nNew York State (2030)')
ax2.title.set_size(9)

plt.show()

In [ ]:
df = pd.concat([g22.portion,g30.portion], axis=1)
df.columns = ["g22","g30"]
capacity_increase = df.g30 / df.g22
capacity_increase[~np.isfinite(capacity_increase)] = g30.portion[~np.isfinite(capacity_increase)]*100
capacity_increase

## Electricity Generation by Month

In [ ]:
monthly_elec = pd.read_csv("data/ny_electricity_by_month.csv", skiprows=4)
monthly_elec.drop(["units","source key"], axis=1,inplace=True)
monthly_elec.set_index("description", inplace=True)
monthly_elec = monthly_elec.T
monthly_elec.drop(["New York", "New York : all fuels (utility-scale)"], axis=1,inplace=True)
monthly_elec.columns = ["hydroelectric", "other_renewables"]

month_years = [i.split("-") for i in monthly_elec.index.to_list()]
months = [i[0] for i in month_years]
years = [int(i[1]) for i in month_years]

monthly_elec.index = months
monthly_elec['year'] = (np.array(years) + 2000).astype(int)

monthly_elec.head(12).T

In [ ]:
hydro_mwh = []
for month in monthly_elec.index.unique():
    hydro_mwh.append(monthly_elec.hydroelectric[monthly_elec.index == month].values)

hydro_mins = pd.DataFrame(hydro_mwh).T.min()
hydro_maxs = pd.DataFrame(hydro_mwh).T.max()
hydro_meds = pd.DataFrame(hydro_mwh).T.median()

fig, ax1 = plt.subplots(figsize=(7.5,3), dpi=110)

parts1 = ax1.violinplot(hydro_mwh, showmedians=False, showextrema=False)
ax1.set_title('Hydroelectric generation by month\n2001-2022')
ax1.set_xticks(np.arange(1, 12 + 1))
ax1.set_ylabel("mWh")
ax1.set_xlabel("month")

for pc in parts1['bodies']:
    pc.set_ec('tab:blue')
    pc.set_facecolor('tab:blue')
    pc.set_alpha(0.5)
    

# ax1.scatter([1]*len(months), monthly_elec.hydroelectric, marker='o', color='k', s=15, alpha=.1,zorder=3)
ax1.scatter(range(1,12+1), hydro_meds, marker='o', color='k', s=30, alpha=1,zorder=3)
ax1.vlines(range(1,12+1), hydro_mins, hydro_maxs, color='k', linestyle='-', lw=0.75)

plt.show()

In [ ]:
other_mwh = []
for month in monthly_elec.index.unique():
    other_mwh.append(monthly_elec.other_renewables[monthly_elec.index == month].values)

other_mins = pd.DataFrame(other_mwh).T.min()
other_maxs = pd.DataFrame(other_mwh).T.max()
other_meds = pd.DataFrame(other_mwh).T.median()

fig, ax1 = plt.subplots(figsize=(7.5,3), dpi=110)

parts1 = ax1.violinplot(other_mwh, showmedians=False, showextrema=False)
ax1.set_title('Solar and wind generation by month\n2001-2022')
ax1.set_xticks(np.arange(1, 12+1))
ax1.set_ylabel("mWh")
ax1.set_xlabel("month")

for pc in parts1['bodies']:
    pc.set_ec('gold')
    pc.set_facecolor('gold')
    pc.set_alpha(0.5)
    
# ax1.scatter([1]*len(months), monthly_elec.hydroelectric, marker='o', color='k', s=15, alpha=.1,zorder=3)
ax1.scatter(range(1,12+1), other_meds, marker='o', color='k', s=30, alpha=1,zorder=3)
ax1.vlines(range(1,12+1), other_mins, other_maxs, color='k', linestyle='-', lw=0.75)

plt.show()

***

## Understanding DERs in 2022

In [ ]:
cols = [
    'Number of Devices',
    'Rated Electric Generation Per Device (kW)',
    'Primary Fuel',
    'ESS Type',
    'ESS Subtype',
    'Rated Electric Storage Capacity Per Device (kWh)',
    'Equivalent Electrical Storage Capacity Per Device from Cooling Sources (kWh)',
]

short_cols = [
    'quantity',
    'generation',
    'fuel',
    'storage_type',
    'storage_subtype',
    'storage_capacity',
    'cooling_storage_capacity',
]

ders = pd.read_excel('data/nys_der_metrics.xlsx', sheet_name='DE Resources')
ders = ders[cols]
ders.columns = short_cols
ders.head()

In [ ]:
total_gen = ders.generation.sum()
total_electric_storage = ders.storage_capacity.sum()
total_cooling_storage = ders.storage_capacity.sum()

### DER Growth Assumptions 

Per state motor vehicle registrations in 2017, there **4%** of all cars in the US are registered in NYS.

[EEI](https://www.eei.org/News/news/All/eei-projects-26-million-electric-vehicles-will-be-on-us-roads-in-2030#:~:text=The%20number%20of%20EVs%20on,on%20U.S.%20roads%20in%202030.) projects that there will be 26.4M electric cars on the road, meaning 4% of those will be in NYS.

Per [Car and Driver](https://www.caranddriver.com/features/g36278968/best-selling-evs-of-2021/) the number one selling electric car in 2021 was the Tesla Model Y with a battery size of ~80kWhs [source](https://www.caranddriver.com/tesla/model-y/specs).

So let's assume that there's a total of **1,056,000 electric vehicles** (26.4M vehicles * 4% in NYS) with a total storage capacity of 84,480,000 kWhs or 84,480 MWhs of ***additional*** electric storage capacity will be available in 2030.

Let's double the adoption of cooling storage...

In [ ]:
# All in MWh
der_stats = [
        {'year': 2022, 'elec_gen': total_gen/1000, 'elec_storage': total_electric_storage/1000, 'cooling_storage': total_electric_storage/1000},
        {'year': 2030, 'elec_gen': total_gen*2.5/1000, 'elec_storage': (total_electric_storage/1000) + 84480, 'cooling_storage': total_electric_storage*2/1000}
    ]

print("DER Capacities by type")
der_stats = pd.DataFrame(der_stats)
der_stats.set_index("year", inplace=True)
der_stats

In [ ]:
print("Operational DER Resources (2022)\n")
print("Total electric generation (MW): {}".format(total_gen/1000))
print("Total electric storage (MWh):    {}".format(total_electric_storage/1000))
print("Total cooling storage (MWh):     {}".format(total_electric_storage/1000))

print("\n\nEstimated DER Resources (2030)\n")
print("Total electric generation (MW): {}".format(total_gen*2.5/1000))
print("Total electric storage (MWh):   {}".format((total_electric_storage/1000) + 84480))
print("Total cooling storage (MWh):     {}".format(total_electric_storage*2/1000))

der_stats.plot.bar(zorder=5)
plt.yscale('log') # or "log"
plt.ylabel("MWh (log scale)")
plt.grid(which='major', zorder=3, color='k', linewidth=1) # color='r', linestyle='-', linewidth=2
plt.grid(which='minor', zorder=4, linewidth=0.5)
plt.title("Expected growth in NY State DERs\n2022 - 2030")
plt.show()

## Simulating a dymanic electricity market

To raise the stakes that **ScOWL** must content with, we'll have an aggregate supply that is always chasing the demand curve. In order to prevent blackouts, and minimize brownouts, we'll have to rapidly recalcuate the *ephermal distribution trees* for each `Generateor` presently available.

#### In short,

- When we have an excess of supply, our storage resources should kick in.
- When we have an too much demand, a `Generator` should signal to each `Consumer` in their distribution tree to reduce demand (via a `low_capacity` RPC).
- If a `Consumer` does not back off fast enough, it could blackout their whole tree.
- When a `Consumers` region is blackedout, the `Tracker` tries to pair its consumption with another `Generator` and add it to their tree. If it cannot find a `Generator` with available capacity. That `Consumer` gets ***starved***. 
- `Consumer` starvation is a key metric for evaluation.

In [ ]:
# T is a multiplier that is used to set the timescale of each trial run.
# T may be 12 seconds --> 12 months.
# The base units are seconds
T = 12
T_units = "seconds"
time = np.arange(0,T*np.pi,0.1)
supply = np.cos(time) + g30.mwh.sum()
demand = np.sin(time) + g30.mwh.sum()
data = {
    "supply": supply,
    "demand": demand
}
df = pd.DataFrame(data, index=time)

fig, ax1 = plt.subplots(figsize=(7.5,3), dpi=110)

ax1.plot(df['supply'], color="black")
ax1.plot(df['demand'], color="tab:red")            
ax1.set_ylabel("MWh")
ax1.set_xlabel(T_units)
ax1.set_title("Available Supply vs. Desired Demand")
plt.legend(["supply","demand"])

plt.show()

## How many generators are online today?

State codes:

In [ ]:
pd.DataFrame([["NY", 36],["PA", 42],["MA", 25],["NJ", 34]], columns=("state","code")).set_index("state").T

In [ ]:
gen_facilities = pd.read_csv('data/2021_generation_facilities.csv',skiprows=1)
gen_facilities.head(3)

In [ ]:
# many hydro facilities produced no electricity in 2021, maybe they are old?
fig = gen_facilities[gen_facilities.gwh > 0]["Type 1"].value_counts().plot.bar()
fig.set_title("NYISO Facility Counts (2021)")
fig.set_ylabel("number of facilities")
fig.set_xlabel("generation type")
plt.show()

In [ ]:
fig = gen_facilities.groupby("Type 1").sum()["Name Plate Rating (V) MW"].plot.bar()
fig.set_title("NYISO Facility Nominal Capacity (2021)")
fig.set_ylabel("output (MW)")
fig.set_xlabel("generation type")
plt.show()

In [ ]:
fig = gen_facilities.groupby("Type 1").sum()["gwh"].plot.bar()
fig.set_title("NYISO Facility Generation (2021)")
fig.set_ylabel("GWh")
fig.set_xlabel("generation type")
plt.show()

### Average output by facility type

In [ ]:
num_facilities = gen_facilities[gen_facilities.gwh > 0]["Type 1"].value_counts()
nominal_capacity = gen_facilities.groupby("Type 1").sum()["Name Plate Rating (V) MW"]
avg_facility_cap = nominal_capacity / num_facilities
avg_facility_cap = np.round(avg_facility_cap[avg_facility_cap > 0]).astype(int)
avg_facility_cap

***

# Testbed Parameters

## Part 1: Instantiating peers

### Generators

Drawing from the stats above, we will instate generators proportional to:
1. Total generation capacity
2. Generation capacity by unit type
3. Number of units as function of average unit size

Per the 2021 "Gold Book" the forecasted total capacity is expected to be: **41,071 MW**

Per the 2021 "Gold Book" the forecasted Summer Peak Demand to be: **32,121 MW**

Per the 2021 "Gold Book" the forecasted Winter Peak Demand to be: **25,275 MW**


In [ ]:
# total capacity (MW)
nominal_cap = 41071
# Summer peak demand (MW)
summer_peak = 32121
# Winter peak demand (MW)
winter_peak = 25275

In [ ]:
std_index_counts = {
    "hydroelectric": ["WAT"],
    "nuclear": ["UR"],
    "natural gas-fired": ["NG","MTE"],
    "utility-scale solar": ["SUN"],
    "land-based wind": ["WND"],
    "petroleum-fired": ["BIT","KER","FO2","FO6","REF"]
}

In [ ]:
# # uncomment to view exact number of facilities, by sub-type
# gen_facilities[gen_facilities.gwh > 0]["Type 1"].value_counts()

In [ ]:
def sumAndReindex(data, indexes):
    reformatted_data = []
    for t in indexes:
        reformatted_data.append([t, data[indexes[t]].sum()]) 
    return reformatted_data

facil_counts = sumAndReindex(gen_facilities[gen_facilities.gwh > 0]["Type 1"].value_counts(), std_index_counts)
facil_counts = pd.DataFrame(facil_counts, columns=["type","counts"]).set_index("type")
print("True Count of Generation Facilities by Type")
facil_counts.T

In [ ]:
def avgAndReindex(data, indexes):
    reformatted_data = []
    for t in indexes:
        reformatted_data.append([t, data[indexes[t]].mean()]) 
    return reformatted_data

avg_gen = avgAndReindex(avg_facility_cap, std_index_counts)
avg_gen = pd.DataFrame(avg_gen, columns=["type","capacity_mw"]).set_index("type")
print("Average Nominal Generation Facility Capacity (MW)")
avg_gen.T

## Compute number of generation facilities to instantiate 

\<facility_type\><sub>total</sub> = total_generation<sub>nominal</sub> / generation_per_facility<sub>average</sub>

In [ ]:
# add better notes HERE for parametric facilities counts
facil_counts = np.round((nominal_cap * g22.portion / avg_gen.T))
facil_counts = facil_counts.T.sort_values('capacity_mw', ascending=False)
facil_counts.columns = ["counts"]
facil_counts.T

In [ ]:
plt.bar(facil_counts.index, facil_counts.counts.values, color=colors.color[facil_counts.index].values)
plt.title("2022 Generator Counts")
plt.ylabel("number of generators")
plt.xticks(rotation=90)
plt.show()

In [ ]:
future_facil_counts = np.ceil(facil_counts.T * capacity_increase).T
future_facil_counts[np.isnan(future_facil_counts.counts)] = 0
future_facil_counts.counts = future_facil_counts.counts.astype(int)
future_facil_counts.T

In [ ]:
future_facil_counts.loc[facil_counts.index]

In [ ]:
plt.bar(
    future_facil_counts.loc[facil_counts.index].index,
    future_facil_counts.loc[facil_counts.index].counts,
    color=colors.color[future_facil_counts.loc[facil_counts.index].index].values)
plt.title("2030 Generator Counts")
plt.ylabel("number of generators")
plt.ylim(0,250)
plt.xticks(rotation=90)
plt.show()

In [ ]:
final_gen_stats = pd.concat([facil_counts, avg_gen], axis=1)
print("We'll instantiate our testbed with these parameters:")
final_gen_stats.T

***

### Consumers

The most recently published dataset is from 2018. We'll base our portion of consumers based on that.

In [ ]:
elec_customers = pd.read_csv('data/electricity_sales_by_customer_GWh.csv').set_index("Year").drop("Grand Total", axis=1)
portion_elec_customers = elec_customers.loc[2018] / elec_customers.loc[2018].sum() 

In [ ]:
plt.pie(portion_elec_customers, labels=portion_elec_customers.index, autopct='%.1f%%')
plt.title("Portion of Electricity Use by Customer Type")
plt.show()

According to [Census.gov](https://www.census.gov/quickfacts/NY), there are **8,531,063 housing units** in New York State, as of 2021.

According to the [NYSERDA commercial baseline study](https://www.nyserda.ny.gov/about/publications/building-stock-and-potential-studies/commercial-statewide-baseline-study), there are **367,223 commercial buildings** in New York State. (Vol 1., pg.13)

For simplicity, we'll assume that **transportation** and **industrial** end uses have ***fixed*** energy consumption.

In [ ]:
res_units = 8531063 
com_units = 367223

In [ ]:
# Average consumption by unit type (MW)
consump_by_cust_type = summer_peak * portion_elec_customers
consump_by_cust_type

In [ ]:
res_unit_consump = consump_by_cust_type["Residential"] / res_units
com_unit_consump = consump_by_cust_type["Commercial"] / com_units
print("Residential Units:          {}".format(res_units))
print("Avg. res unit consumption:  {} MW".format(np.round(res_unit_consump, 5)))
print("Commercial Units:           {}".format(com_units))
print("Avg. com. unit consumption: {} MW".format(np.round(com_unit_consump, 5)))

We'll use this [source ](https://www.perchenergy.com/blog/energy/what-appliances-use-most-electricity-home)for energy break downs for residential units. TV has been combined with plug loads.

In [ ]:
res_elec_by_use = [
    {"load": "hvac", "portion": 0.45, "deferrable_for": 60*10},
    {"load": "hot_water", "portion": 0.12, "deferrable_for": 60*5},
    {"load": "lighting", "portion": 0.10, "deferrable_for": 0},
    {"load": "fridge", "portion": 0.08, "deferrable_for": 60*3},
    {"load": "washer_dryer", "portion": 0.05, "deferrable_for": 60*15},
    {"load": "oven", "portion": 0.03, "deferrable_for": 0},
    {"load": "dishwasher", "portion": 0.02, "deferrable_for": 60*30},
    {"load": "other", "portion": 0.15, "deferrable_for": 60*30},
]

In [ ]:
print("Prototypical energy loads (residential)")
res_elec_by_use = pd.DataFrame(res_elec_by_use)
res_elec_by_use

***

In [ ]:
res_elec_by_use.portion.sum()

# Trackers

Trackers must maintain *ephemeral distribution trees*, graphs that connect a `Consumer`  to a `Generator` with  spare capacity. As `Consumer` demand increases, `Generators` may be forced to `disconnect()` one or more consumers to maitain a `safetyThreshold`.

The following is a sketch of the tracker microservice.

In [ ]:
class Tracker:
    def __init__(self):
        self.Trees = []
        self.Consumers = []
        self.Generators = []
    
    def balanceTrees(self):
        """Balance all trees managed by the tracker"""
        pass
    
    def findGenerator(self, consumer):
        """Find a generator with spare capacity for a consumer to matched with"""
        pass
    
    def findConsumers(self, generator):
        """Find a consumer with to absorb spare generation capacity"""
        pass
    
    def shedLoad(self, generator):
        """Pick consumers to disconnect from a generator to maintain the safety threshold"""
        pass
    
    def registerGenerator(self, generator):
        """Add a generator to self.Generators"""
        pass
    
    def registerConsumer(self, consumer):
        """Add a consumer to self.Consumers"""
        pass
    

Trackers must store metadata about each `Generator` and `Consumer` that they are responsible for. The abbreviated set of `Peer` metadata the tracker must store is captured below:

In [ ]:
SEED = 42 # for deterministic hashing with MurmurHash3

class mPeer:
    def __init__(self, addr):
        self.addr = addr #IPV4 address
        self.id = self.setHash()
    
    def getAddr(self):
        return self.addr
    
    def setHash(self):
        import mmh3
        import secrets
        import warnings
#         warnings.filterwarnings('ignore')
        self.id = mmh3.hash(self.addr + secrets.token_hex(2),SEED) # secrets.token_hex is non-deterministic...
        return self.id

class mGenerator(mPeer):
    def __init__(self, addr, nominal_capacity, kind):
        self.addr = addr #IPV4 address
        self.id = self.setHash()
        
        self.kind = kind
        self.nominal_cap = nominal_capacity # MW
        
        self.cur_cap = 0    # MW
        self.cur_demand = 0 # MW

class mConsumer(mPeer):
    def __init__(self, addr, avg_consump, kind, desired_consump=None, deferrable_consump=0, demand_responsive_consump=0):
        self.addr = addr #IPV4 address
        self.id = self.setHash()
        
        self.avg_consump = avg_consump               # MW
        self.deferrable_consump = deferrable_consump # MW
        self.dr_consump = demand_responsive_consump  # MW
        
        if desired_consump is None:
            self.desired_consump = self.avg_consump  # MW
        
        self.cur_consump = 0 # current consumption in MW        

class mStorage(mConsumer):
    def __init__(self, addr, capacity):
        self.addr = addr  #IPV4 address
        self.id = self.setHash()

        self.capacity = 0 # MW

In [ ]:
# Test
# this will return a new value each time because of the use of the non-determinstic nonce...
# I don't think that's ideal.
p = mPeer("192.168.1.2:3003")
print(p.id)

## Assigning Buckets for Generator Trackers

To evenly distribute the labor of generating distribution trees. We'll need to assign a fixed number of buckets to filter each `Generator.id` into.

I'll be assuming the use of the d430 resource type on emulab. It has:
- 16 CPU cores @ 2.4Hz
- 64 GB DDR4 RAM @ 2133MT/s

Accordingly, we'll assume **8-16 `Tracker` buckets**.

### Method

Bucket assignment works as follows:

In [ ]:
HASH_SIZE = 32

In [ ]:
def assignBucket(hash_result, num_buckets=8):
    # map hash_result in to the range [0,1]
    hash_result += 2**(HASH_SIZE-1)
    p = hash_result / 2**(HASH_SIZE)
    b = int(np.round(p * (num_buckets)))
    if b >= num_buckets: b = 0 
    return b

### Checking for correctness

In [ ]:
r = range(-1 * 2**(HASH_SIZE-1), 2**(HASH_SIZE-1) + 1, 2**16)
bucket = []
for sample in r:
    bucket.append(assignBucket(sample))

In [ ]:
results = pd.DataFrame(bucket)

In [ ]:
pd.DataFrame(results.value_counts(),columns=["bucket_size"]).sort_index().T

## Tracker Mock Up

How do Trackers get their `Generator` assignments?

- a loadbalancer?

What's the use case (bootstrapping)?
1. New `Generator` comes online.
2. It pings a server to know where which `Tracker` it should be assigned to.
3. When that server responds, the `Geneterator` connects with that `Tracker` so it may be given an ephemeral distribution tree.

### Bootstrapping Server Mockup

In [ ]:
# a new generator comes online,
# it pings the bootstrapping server for the tracker_id and ip_address to reach out to:
class GeneratorBootstrappingServer():
    def __init__(self):
        self.trackers = {
            1 : "192.168.1.1:3001",
            2 : "192.168.1.1:3002",
            3 : "192.168.1.1:3003",
            4 : "192.168.1.1:3004",
            5 : "192.168.1.2:3001",
            6 : "192.168.1.2:3002",
            7 : "192.168.1.2:3003",
            8 : "192.168.1.2:3004",
        }
        self.HASH_SIZE = 32 # if using mmh3.hash()
    
    def assign_bucket(self, gen_addr, num_buckets=8):
        # add some a random nonce
        import mmh3
        import secrets
        import warnings
        # hashing done at the bootstrapping server so that a compromised generator can't 
        # intentionally overload a tracker by trying to register generators with the same ip and "nonce"
        # secrets.token_hex is non-deterministic...
        hash_result = mmh3.hash(gen_addr + secrets.token_hex(2),SEED)
        
        # get the hash_result into the range of an unsigned 32-bit int
        hash_result += 2**(self.HASH_SIZE-1)

        # map hash_result in to the range [0,1]
        p = hash_result / 2**(self.HASH_SIZE)
        
        # assign to specific bucket
        b = int(np.round(p * (num_buckets)))
        if b >= num_buckets: b = 0 # clean up results
        
        # return the bucket to the client (Generator)
        return b

In [ ]:
gbs = GeneratorBootstrappingServer()

Sanity Check

In [ ]:
peers_to_sort = 1000



### Correctness Check No. 2

In [ ]:
generator_mdata = []
for kind in final_gen_stats[final_gen_stats.counts > 0].index:
    for facil in range(final_gen_stats.loc[kind].counts.astype(int)):
        generator_mdata.append(
            mGenerator(
                '192.168.1.0:65535',
                final_gen_stats.loc[kind].capacity_mw,
                kind
            )
        )
    print("Created: {} generators".format(kind))

print("\nTotal Generators created: {}".format(len(generator_mdata)))

In [ ]:
# Create bootsctrapping server
gbs = GeneratorBootstrappingServer()

tracker_assignments = []
for generator in generator_mdata:
    tracker_assignments.append([gbs.assign_bucket(generator.addr)])


pd.DataFrame(tracker_assignments)[0].value_counts().sort_index().plot.bar()
plt.show()

***

## Memory allocation test

Let's see how much memory it takes to store the basic data about each peer.

Before instantiation of `Generators` this notebook has used **254.4 MB** of memory.

We'll now instantiate the following number of `Peers`, with dummy data.

In [ ]:
final_gen_stats.T

In [ ]:
generator_mdata = []
for kind in final_gen_stats[final_gen_stats.counts > 0].index:
    for facil in range(final_gen_stats.loc[kind].counts.astype(int)):
        generator_mdata.append(
            mGenerator(
                '192.168.1.0:65535',
                final_gen_stats.loc[kind].capacity_mw,
                kind
            )
        )
    print("Created: {} generators".format(kind))

print("\nTotal Generators created: {}".format(len(generator_mdata)))

In [ ]:
# Inspect sample
print("type:    ", generator_mdata[256].kind)
print("addr:    ", generator_mdata[256].addr)
print("capacity:", generator_mdata[256].nominal_cap, "MW")

After instantiation of 342 `Generators` this notebook has used **254.4 MB** of memory. So, holding all generators in memory requires **3.2 MB** of memory

Before instantiation of `Consumers` this notebook has used **273.2 MB** of memory.

We'll now instantiate the following number of `Peers`, with dummy data.


In [ ]:
print("Residential Units:          {}".format(res_units))
print("Avg. res unit consumption:  {} MW".format(np.round(res_unit_consump, 5)))
print("Commercial Units:           {}".format(com_units))
print("Avg. com. unit consumption: {} MW".format(np.round(com_unit_consump, 5)))

In [ ]:
final_consumer_stats = pd.DataFrame(
    [
        {"counts": res_units, "avg_consump": res_unit_consump},
        {"counts": com_units, "avg_consump": com_unit_consump}
    ],
    index=["residential", "commercial"]
)
final_consumer_stats

In [ ]:
consumer_mdata = []
consumers_created = 0
for kind in final_consumer_stats[final_consumer_stats.counts > 0].index:
    print(kind)
    for consumer in range(final_consumer_stats.loc[kind].counts.astype(int)):
        consumer_mdata.append(
            mConsumer(
                '192.168.1.0:65534',
                final_consumer_stats.loc[kind].avg_consump,
                kind
            )
        )
        consumers_created += 1
        if consumers_created >= 10000: break

In [ ]:
print("Created: {} consumers".format(len(consumer_mdata)))

In [ ]:
mem_per_consumer = np.round(( 493.2 - 254.4 ) / 1000002, 8)
mem_per_consumer

Before instantiation of `Consumers` this notebook has used **254.4 MB** of memory.

After instantiation of `Consumers` this notebook has used **493.2 MB** of memory.

So the average memory per consumer is: **0.0002388 MB**


So, to manage all the `Consumers` it would require:

In [ ]:
print(final_consumer_stats.counts.sum() * mem_per_consumer, "MB of memory")

Or, about **2GB**

***

## Energy Mix Generation Scripts

We need to start by instantiating some peers. We'll do this at a fixed ratio of `Consumer`s and `Generator`s, **20,000:1**.

In [ ]:
CTG = 20000/1

In [ ]:
num_generators = 1 # multiplies the CTG rate to increate the number of consumers and generators

In [ ]:
future_facil_counts[future_facil_counts == 0] = 1
future_facil_portions = pd.DataFrame(future_facil_counts, index=facil_counts.index)
future_facil_portions.columns = ["2030_portions"]
future_facil_portions["2030_portions"] = future_facil_portions["2030_portions"] / future_facil_portions["2030_portions"].sum()
future_facil_portions.T

In [ ]:
# uncomment to save these portions,
# then relocate from disk 
portions_path = 'simulation_config/2030_energy_mix/portions.csv'
future_facil_portions.to_csv(portions_path)
future_facil_portions = pd.read_csv(portions_path).set_index("type")
future_facil_portions.T

This function computes the **number** and **types** of generation facilities to instantiate for the trial.

In [ ]:
def getRandFacility(_df, num_facilities=1, seed=10):
    import numpy as np
    """you must pass dataframe with facility types as the index and the portions as the first column"""
    
    # format incoming data
    df = _df # we want a copy
    col_0 = df.columns[0]
    df.sort_values(col_0, ascending=False, inplace=True)
    portions = df[col_0]
    
    # compute breakpoints
    df["breakpoint"] = [np.sum(portions[:i+1]) for i, _ in enumerate(portions.values)]
    
    # create place to store output
    output = df.index

    # use a deterministic PRN
    np.random.seed(seed) # default is 10
    
    # compute the facility determined by the result and the breakpoint
    df["counts"] = [0] * len(df)
    for i in range(num_facilities):
        result = np.random.random()
        for kind in df.index:
            if result <= df.loc[kind].breakpoint:
                df.at[kind,"counts"] =  df.at[kind,"counts"] + 1
                break
    return df

Change this to alter the number of generators in the simulation

In [ ]:
num_generators = 100

In [ ]:
gen_to_instantiate = getRandFacility(future_facil_portions, num_generators) # set above
gen_to_instantiate = pd.concat([gen_to_instantiate, avg_gen], axis=1)
gen_to_instantiate.at["offshore wind", "capacity_mw"] = gen_to_instantiate.at["land-based wind", "capacity_mw"]
#  home solar outout = 250 W per panel * 25 panels / 1000000 W per MW
gen_to_instantiate.at["distributed solar", "capacity_mw"] = ((250 * 25) / 10**6)
gen_to_instantiate["total_output"] = gen_to_instantiate["counts"] * gen_to_instantiate["capacity_mw"]

print("Total Capacity (nominal) of {} Generators: {} MW".format(num_generators, gen_to_instantiate.total_output.sum()))
gen_to_instantiate.drop(["2030_portions","breakpoint"],axis=1).T

https://www.cnet.com/home/energy-and-utilities/find-out-how-many-solar-panels-you-need-to-power-your-house/

Let's investigate the output visually.

In [ ]:
from matplotlib.ticker import MaxNLocator

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7.5,4), dpi=110)
fig.suptitle('Trial Energy Mix: Facility Count and Output (MW)')

_index = gen_to_instantiate.index

ax1.bar(_index, gen_to_instantiate.counts.astype(int), color=colors.color[_index].values)
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))
ax1.set_ylabel("number of facilties")
ax1.tick_params(axis='x', labelrotation=90)

ax2.set_ylabel("total output (MW)")
ax2.bar(_index, gen_to_instantiate.total_output, color=colors.color[_index].values)
ax2.tick_params(axis='x', labelrotation=90)

print("total generation facilities: {}".format(gen_to_instantiate.counts.sum()))
print("total generation capacity:   {:.0f} MW".format(gen_to_instantiate.total_output.sum()))

plt.tight_layout()
plt.show()

Now we need to take this and output a config file or series of config files to be instantiated on emulab

In [ ]:
def FormatGenConfigFiles(frame, num_hosts=1):
    """Takes two arguments a dataframe of the number of facilities
    and the number of config files you'd like to generate (one for each host).
    """
    columns = ['counts','capacity_mw','total_output']
    df = frame[columns] # make a copy
    # to load balance the trackers we should allocate a roughly even total_output to each tracker
    sorted_df = df.sort_values('capacity_mw', ascending=False)
#     print(sorted_df)
    
    capacity_per_host = df['total_output'].sum() / num_hosts
    print()
    print("Config files will include:")
    print("    - {} generators sharded across {} hosts".format(df['counts'].sum(), num_hosts))
    print("    - Each host will be responsible for ~{:.2f} MW".format(capacity_per_host))
    print("    - For a total of {} MW of generation capacity".format(df['total_output'].sum()))    
    print()
    
    facil_remaining = sorted_df.copy()
    print("\nGeneration to allocate")
    print("---------")
    print(facil_remaining)
    print("---------")
#     print(pd.DataFrame(facil_remaining.sum(),columns=['Total']).set_index('Total',inplace=True).T)
#     print("---------")

    # copy df and reset mutable values to zero 
    initial_df = sorted_df.copy()
    initial_df['counts'] = 0
    initial_df['total_output'] = 0
    
    # create a list containing a dataframe for each host.
    host_responsibilities = [initial_df.copy() for i in range(num_hosts)]
    
    # iterate through the kinds of facilities,
    # starting with the ones with the highest output
    print()
    for kind in facil_remaining.index:
#         print("There were {} {} generators, ".format(facil_remaining.at[kind,"counts"], kind), end='')
        while (facil_remaining.at[kind,"counts"] > 0):
            for i in range(num_hosts):
                host_responsibilities[i].at[kind,"counts"] += 1
#                 print(host_responsibilities[i].at[kind,"counts"], end=" ")
#                 print(facil_remaining.at[kind,"counts"])
                facil_remaining.at[kind,"counts"] -= 1
                if (facil_remaining.at[kind,"counts"] == 0): break
#         print("now there are {} {} generators".format(facil_remaining.at[kind,"counts"], kind))

    for i, host in enumerate(host_responsibilities):
        host_responsibilities[i]['total_output'] = host_responsibilities[i]['counts'] * host_responsibilities[i]['capacity_mw']
        print("\nHost {}".format(i+1))
        print("---------")
        print(host)
    
    return host_responsibilities
    
host_configs = FormatGenConfigFiles(gen_to_instantiate, num_hosts=1)
host_configs

host_facils = 0
host_output = 0

print("---------")
print("\nHost allocation summary")
print("---------")
for i, h in enumerate(host_configs):
    host_facils += h.counts.sum()
    host_output += h.total_output.sum()
    print("Host {}:".format(i+1))
    print("    - Facilities {}".format(h.counts.sum()))
    print("    - Capacity   {} MW".format(h.total_output.sum()))
print("=========")
print("Simulation total:")
print("    - Facilities {}".format(host_facils))
print("    - Capacity   {} MW".format(host_output))
print("---------")

# print()
# print("resulting facilities: {}".format(host_facils))
# print("resulting output:     {} MW".format(host_output))

Next we need to take this and generate a config file...
***
### Writing Config Files
Let's write each dataframe to a CSV and try loading it from a standalone program.

In [ ]:
for i, h in enumerate(host_configs):
    path = 'simulation_config/2030_energy_mix/host_configs/host_config_{}.csv'.format(i+1)
    h.to_csv(path)

### Loading from config & adding address
When the generator bootstraps it will need to provide its **IP address** and the **port** it would like to be contacted at.

We'll load the `generator`s from config folder.

In [ ]:
import os

config_path = 'simulation_config/2030_energy_mix/host_configs/'
config_files = [f for f in os.listdir(config_path) if os.path.isfile(join(config_path, f))]
config_files

In [ ]:
config_from_disk = []
for file in config_files:
    full_path = os.path.join(config_path, file)
    print(pd.read_csv(os.path.join(config_path, file)).set_index('type'))

In [ ]:
def MakeGenInstances(ref_df, starting_port=32000, latency_best_case=10, latency_worst_case=100):
    """takes a data frame of counts and capacities,
    returns a dataframe with IP addresses and ports.
    
    Latency is in ms
    """
    import socket
    ip_addr = socket.gethostbyname(socket.gethostname())
    
    num_rows = ref_df['counts'].sum()
    
    # intialize a empty dataframe
    _df = pd.DataFrame()
    gen_types = []
    gen_capacity = []
    for kind in ref_df.index:
        for i in range(int(ref_df.at[kind, 'counts'])):
            gen_types.append(kind)
            gen_capacity.append(ref_df.at[kind, 'capacity_mw'])
    _df['type'] = gen_types
    _df['capacity_mw'] = gen_capacity
    
    # assign IP and ports to each generator
    _df['addr'] = ["{}:{}".format(ip_addr, starting_port+i) for i in range(num_rows)]
    # add a latency from 10 ms to 100 ms (uniformly distributed)
    _df['rtt'] = np.random.randint(latency_best_case, latency_worst_case+1, num_rows)
    
    return _df

MakeGenInstances(pd.read_csv(os.path.join(config_path, config_files[0])).set_index('type'),
                 starting_port=32000)

__12/6/2022 21:22__

At this point I'm able to generate all the necessary metadata to instantiate all generators on a host

__TODO:__
- move to seperate client program. 
- have that client program launch instances of a generatorRPC clients
- have those instances request hashes

Let's templatize the writing of a bashscript. Then modify the access priviledges of the file.

In [ ]:
def AppendLine(s, line):
    """takes a base string, s, and appends line on a newline"""
    return "{}\n{}".format(s,line)

In [ ]:
# for each generator in the trial
# we'll instantiate one version of the generator client
generator_script = "#!/usr/bin/env bash" # this will need to be changed for ubuntu

lines = []
generator_script = AppendLine(generator_script, "echo 'hellow world!'")

In [ ]:
gen_launcher_path = 'simulation_config/2030_energy_mix/gen_launcher.sh'
with open(gen_launcher_path, 'w') as reader:
    reader.write(generator_script)

subprocess.run(["cat", gen_launcher_path])

In [ ]:
# change it to an executable
subprocess.run(["chmod", "+xu", gen_launcher_path])

In [ ]:
# test it
with open('logs/bootstrap_server.log', 'a') as f:
    subprocess.run(["./{}".format(gen_launcher_path)], stdout=f)

In [ ]:
subprocess.run(["./{}".format(gen_launcher_path), ">>", "logs/bootstrap_server.log"])


In [ ]:
import time

# bootstrap_server_proc = subprocess.Popen(["python","bootstrap_server.py"])
# time.sleep(15) # ms
bootstrap_server_proc.terminate()

***

Next we'll instantiate our consumers at `Consumer` to `Generator`, **20,000:1**.

In [ ]:
num_consumers = round(CTG * num_generators)
print("Generators: {}".format(num_generators))
print("Consumers:  {:,}".format(num_consumers))

In [ ]:
def getConsumers(avg_res_consump, _num_consumers, consumers_types=["Residential"]):
    """returns a dataframe of the instantiation data for the consumers in a given simulation"""
    consumers_to_initialize = pd.DataFrame([[np.round(_num_consumers).astype(int), avg_res_consump]], index=consumers_types, columns=["num_consumers","avg_consump"])
    consumers_to_initialize["total_consump"] = consumers_to_initialize["num_consumers"] * consumers_to_initialize["avg_consump"]
    return consumers_to_initialize

consumers_to_initialize = getConsumers(res_unit_consump, num_consumers)
consumers_to_initialize

In [ ]:
print("Simulation Parameters: ")
print("Consumption: {} MW".format(consumers_to_initialize["total_consump"].loc["Residential"]))
print("Generation:  {} MW".format((gen_to_instantiate["counts"] * gen_to_instantiate["capacity_mw"]).sum()))

***

In [ ]:
# we'll pull a generator at random from this distribution
future_facil_counts = np.round(facil_counts.T * capacity_increase).astype(int).T.counts.values
plt.bar(facil_counts.index, future_facil_counts, color=colors.color[facil_counts.index].values)
plt.title("2030 Generator Counts:\n{} Facilities".format(sum(future_facil_counts)))
plt.ylabel("number of generators")
# plt.ylim(0,1)
plt.xticks(rotation=90)
plt.show()

## Hashing into uniform buckets

To evently distribute the labor of matching a `Generators` with a set `C` of consumers, we need to generate hashes uniformly and distributed over the range [0,2<sup>n</sup>). In the case of MurmurHash3, that's a 32 bit range.

For reproduceability, the hash should be deterministic, so using a specified seed is important.

In [ ]:
import warnings
warnings.filterwarnings(action='once')

In [ ]:
import mmh3

In [ ]:
print(mmh3.hash('foo', 42))
print(mmh3.hash('bar', 42))
print(mmh3.hash('foo', 42))

In [ ]:
128/8

In [ ]:
a = mmh3.hash('foo', 42)
a.to_bytes(4, "big", signed=True).decode('unicode_escape')

***

# gRPC

We'll be defining the **services** and **types** used by those services.

Let's start by defining the different services each node type will need to support:

## Generators

A `generator` must be able to request being added to the network; declare to the network what its type; genertaion capacity (in MWs) and its safety threshold (% of total capacity), be made aware of its tracker, update its tracker when its capacity has changed; request that the tracker shed a minimum amount of load (by reassigning consumers to other generators); hold the state of  demand on itself (by keeping an associative array of active consumers and their consumption); "black out" if demand from the consumers in its *distribution trees* exceeds its current generation capacity; track the time and duration of blackouts.

Black outs occur when a `tracker` is too slow to reassign excess demand on a `generator`. During a black out all consumers in the `generator`'s distribution tree are **choked**. A such, a `generator` must store its own copy of its distribution tree, so it can multicast to the `consumer`s that they are being choked.

### Services

#### `GeneratorJoin`
##### Methods:
- `GeneratorJoin(PeerCtx)` &rarr; `IdHash`
- `ConsumerJoin(PeerCtx)` &rarr; `Guid`



## Consumers

A `consumer` must be able to request a 128-bit ID<sup>1</sup>; maintain the state of its `tracker` and `generator` addresses, and its `current_consumption` and its `last_consumption_signalled`; mutate its `current_consumption` pseudo-randomly; update its generator after each call to `mutate_consumption`; update its `tracker` if and only if the change in consumption is above a given threshold or if it is `deferrable`. When the `consumer` signals that it wants to start `deferrable` consumption, it starts a timeout. At timeout the deferred consumption is added to its `generator`.

<sup>1</sup> For interchangability with the devices IPv4 address, we'll use 128-bit random int bit shifted 80 + 16 bits to the right, so we are left with a 32-bit address in the least significant digits. In the future, we may just use the real consumers IPv6 address.

## Tracker

A `tracker` must be able to maintain state about the generators assigned to it, the consumers assigned to it (and their `current_consumption`); track the available capacity of all generators managed by a peer tracker; request to pass consumers from their management to a peer tracker with avaiable capacity.

## Bootstrapping Server

The `bootstrap_server` provides IDs to all generators and consumers that request them.

### Services

#### `JoinGridService`
##### Methods:
- `GeneratorJoin(PeerCtx)` &rarr; `IdHash`
- `ConsumerJoin(PeerCtx)` &rarr; `Guid`



## Generator Metronome

the `generator_metronome` server receives a `start` signal once all the peers have been bootstrapped; then it 
multicasts updates to rewnewable generators after each timestep (`t`) has been computed.


***

In [ ]:
# plt.pie(np.round(g30[g30.portion > 0].portion*100,1), labels=g30[g30.portion > 0].index,autopct='%.1f%%')
# plt.title("Electricity generation by source\nNew York State (2030)")
# plt.show()

In [ ]:
# plt.pie(np.round(g22[g22.portion > 0].portion*100,1), labels=g22[g22.portion > 0].index,autopct='%.1f%%')
# plt.title("Electricity generation by source\nNew York State (2022)")
# plt.show()

In [ ]:
hydro_fluctuations = (monthly_elec.hydroelectric.max() - monthly_elec.hydroelectric.min()) / monthly_elec.hydroelectric.max()
print("Hydroelectric generation fluctuates by: {}%".format(round(hydro_fluctuations*100)))

In [ ]:
# Monthly electric generation Hydro
plt.scatter(monthly_elec.index, monthly_elec.hydroelectric)
plt.show()

In [ ]:
other_fluctuations = (monthly_elec.other_renewables.max() - monthly_elec.other_renewables.min()) / monthly_elec.other_renewables.max()
print("Solar and Wind generation fluctuates by: {}%".format(round(other_fluctuations*100)))

In [ ]:
# Monthly electric generation Solar and Wind
plt.scatter(monthly_elec.index, monthly_elec.other_renewables, color="gold")
plt.show()